# 数据预处理

In [2]:
#把城市拆开，一个城市一行
import pandas as pd

# 读取Excel文件
file_path = '全网新闻数据.xlsx'
df = pd.read_excel(file_path)

# 定义一个函数来拆分城市列
def split_cities(row):
    cities = row['城市'].strip("[]").replace("'", "").split(', ')
    return [{'date': row['date'], '新闻内容': row['新闻内容'], '城市': city} for city in cities]

# 应用拆分函数
split_rows = df.apply(split_cities, axis=1).explode().apply(pd.Series)

# 保存拆分后的结果到新的Excel文件
output_file_path = '全网新闻数据1.xlsx'
split_rows.to_excel(output_file_path, index=False)

print(f"拆分后的数据已保存到 {output_file_path}")


<ipython-input-2-07b35750ee17>:17: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  split_rows.to_excel(output_file_path, index=False)


拆分后的数据已保存到 全网新闻数据1.xlsx


In [4]:
#把数字切换为半角
import pandas as pd

# 定义一个函数，将全角字符转换为半角字符
def fullwidth_to_halfwidth(s):
    return ''.join(chr(ord(char) - 0xFEE0) if 0xFF01 <= ord(char) <= 0xFF5E else char for char in s)

# 读取Excel文件
file_path = '全网新闻数据1.xlsx'
df = pd.read_excel(file_path)

# 处理“新闻内容”列，将全角数字转换为半角数字
df['新闻内容'] = df['新闻内容'].apply(lambda x: ''.join(fullwidth_to_halfwidth(char) if '０' <= char <= '９' else char for char in x))

# 保存处理后的数据到新的Excel文件
output_file_path = '全网新闻数据2.xlsx'
df.to_excel(output_file_path, index=False)

print(f"处理后的数据已保存到 {output_file_path}")



<ipython-input-4-a38018823584>:17: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_file_path, index=False)


处理后的数据已保存到 全网新闻数据2.xlsx


## 分词和去除停用词

In [6]:
import pandas as pd
import jieba
import re

# 读取Excel文件
input_file = "全网新闻数据2.xlsx"
output_file = "全网新闻数据3.xlsx"
df = pd.read_excel(input_file)

# 载入用户自定义词典
jieba.load_userdict("user_dict.txt")

# 读取停用词表
with open("stop_words.txt", "r", encoding="utf-8") as file:
    stop_words = set(file.read().splitlines())

# 对第一列进行处理
def clean_and_tokenize(text):
    # 去除特殊字符和标点符号，只保留中文字符
    text = re.sub(r"[^\u4e00-\u9fa5]", "", str(text))
    # 分词
    words = jieba.cut(text)
    # 去除停用词
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

# 对第一列进行处理
df.iloc[:, 1] = df.iloc[:, 1].apply(clean_and_tokenize)

# 保存处理后的结果到新的Excel文件
df.to_excel(output_file, index=False)

print(f"处理后的结果已保存到 {output_file}")


<ipython-input-6-447f7e1175c4>:31: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_file, index=False)


处理后的结果已保存到 全网新闻数据3.xlsx


# 使用大连理工词典进行情感分类

In [7]:
# coding: utf-8
import pandas as pd

# 获取数据集
excel_file_path = '全网新闻数据3.xlsx'
weibo_df = pd.read_excel(excel_file_path)
print(weibo_df.head())


         date                                               新闻内容  城市
0  2012 06 12  中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...  上海
1  2012 06 12  中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...  浙江
2  2012 06 12  中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...  河北
3  2012 06 15  中国台湾 网 月日 消息 台湾 中国时报 报道 岛内 大学教授 假发票 真钱案 滚雪球 愈演...  台湾
4  2012 06 15  上海 试行 启运 港 退税 政策 通知 魇 灾吻 直辖市 计划单列市 财政厅 局 国家税务局...  上海


In [8]:
# coding: utf-8
import pandas as pd


# -------------------------------------情感词典读取-------------------------------
# 注意：
# 1.词典中怒的标记(NA)识别不出被当作空值,情感分类列中的NA都给替换成NAU
# 2.大连理工词典中有情感分类的辅助标注(有NA),故把情感分类列改好再替换原词典中

# 扩展前的词典
df = pd.read_excel('大连理工大学中文情感词汇本体NAU.xlsx')
print(df.head(10))

df = df[['词语', '词性种类', '词义数', '词义序号', '情感分类', '强度', '极性']]
df.head()


    词语  词性种类  词义数  词义序号 情感分类  强度  极性 辅助情感分类  强度.1  极性.1 Unnamed: 10  \
0   脏乱   adj  1.0   1.0   NN   7   2    NaN   NaN   NaN         NaN   
1   糟报   adj  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
2   早衰   adj  1.0   1.0   NE   5   2    NaN   NaN   NaN         NaN   
3   责备  verb  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
4   贼眼  noun  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
5   战祸  noun  1.0   1.0   ND   5   2     NC   5.0   2.0         NaN   
6   招灾   adj  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
7   折辱  noun  1.0   1.0   NE   5   2     NN   5.0   2.0         NaN   
8  中山狼  noun  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
9   清峻   adj  1.0   1.0   PH   5   0    NaN   NaN   NaN         NaN   

   Unnamed: 11  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
6          NaN  
7          NaN  
8          NaN  
9          NaN  


,词语,词性种类,词义数,词义序号,情感分类,强度,极性
0,脏乱,adj,1.0,1.0,NN,7,2
1,糟报,adj,1.0,1.0,NN,5,2
2,早衰,adj,1.0,1.0,NE,5,2
3,责备,verb,1.0,1.0,NN,5,2
4,贼眼,noun,1.0,1.0,NN,5,2


In [9]:
#-------------------------------------七种情绪的运用-------------------------------
Happy = []
Good = []
Surprise = []
Anger = []
Sad = []
Fear = []
Disgust = []

#df.iterrows()功能是迭代遍历每一行
for idx, row in df.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])       
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
    if row['情感分类'] in ['NAU']:     #修改: 原NA算出来没结果
        Anger.append(row['词语'])  

#正负计算不是很准 自己可以制定规则       
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust
print('情绪词语列表整理完成')  
print(Anger)


情绪词语列表整理完成
['忿忿不平', '怒火冲天', '气愤愤', '失落', '愤懑', '悲愤', '勃然大怒', '含怒', '恼羞成怒', '忿怒', '七窍生烟', '发标', '怒容', '吹胡子', '唱老生的生气－吹胡子瞪眼镜', '暴跳如雷', '戟指怒目', '怒形于色', '怒目切齿', '敢怒不敢言', '除恶务尽', '逼上梁山', '阴沉', '急眼', '看扁', '气鼓鼓', '投诉', '令人生气', '没好气', '民怨沸腾', '悲愤交加', '大动肝火', '火气', '怒不可遏', '惹气', '窝火', '光火', '无明火', '耍态度', '触怒', '怒色', '犯肝气', '吃了芋头不下肚－顶心顶肺', '勃然变色', '雷霆之怒', '拍案而起', '切齿愤盈', '我靠', 'kao', 'gun', '吵嘴', '羞愤', '气急败坏', '嗟来之食', '火暴', '气忿', '无名火起', '愤愤不平', '暴怒', '愤怒', '恼火', '气冲冲', '义愤', '挂气', '愤愤', '私愤', '赌气', '横眉', '一刀两段', '一气一个死', '火箭筒射击－两头冒火', '怫然作色', '怒气冲冲', '义愤填膺', '上等轮胎－有气难出', '睚眦', '可气', '当场出彩', '针锋相对', 'TNND', '滚出去', '哇靠', '郁怒', '疾言厉色', '火冒三丈', '义形于色', '忿忿', '抗议', '面有愠色', '气忿忿', '割席断交', '天怒人怨', '悲愤填膺', '郁愤', '纳鞋底戳了手－真气人', '愤世嫉俗', '愤然', '恼怒', '气愤', '震怒', '动肝火', '悻然', '负气', '横眉怒目', '奇耻大辱', '草场着火－火气冲天', '怒气冲天', '气呼呼', 'MD', '真TM', '你丫', '嗔怒', '叱呵', '不甘示弱', '寒碜', '抱不平', '来气', '气汹汹', '竟敢', '愤激', '气不忿', '忧愤', '愤慨', '民愤', '怒视', '易怒', '动气', '大发雷霆', '公愤', '泄愤', '怒目', '一刀两断', '三尸暴跳，七窍生烟', '皇上拍桌子－盛怒', '艴然不悦'

In [10]:
import pandas as pd

# ---------------------------------------情感计算---------------------------------
def emotion_caculate(text):
    positive = 0
    negative = 0
    anger = 0
    disgust = 0
    fear = 0
    sad = 0
    surprise = 0
    good = 0
    happy = 0

    wordlist = text.split()  # 直接使用空格分割文本
    wordset = set(wordlist)
    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive += freq
        if word in Negative:
            negative += freq
        if word in Anger:
            anger += freq
        if word in Disgust:
            disgust += freq
        if word in Fear:
            fear += freq
        if word in Sad:
            sad += freq
        if word in Surprise:
            surprise += freq
        if word in Good:
            good += freq
        if word in Happy:
            happy += freq

    emotion_info = {
        'length': len(wordlist),
        'positive': positive,
        'negative': negative,
        'anger': anger,
        'disgust': disgust,
        'fear': fear,
        'good': good,
        'sadness': sad,
        'surprise': surprise,
        'happy': happy,
    }

    indexs = ['length', 'positive', 'negative', 'anger', 'disgust', 'fear', 'sadness', 'surprise', 'good', 'happy']
    return pd.Series(emotion_info, index=indexs)

# 读取新的文本数据
excel_file_path = '全网新闻数据3.xlsx'
news_df = pd.read_excel(excel_file_path)

# 测试情感计算函数
for index, row in news_df.iterrows():
    text = row['新闻内容']  # 假设文本列的列名是"文本"
    res = emotion_caculate(text)
    print(res)


length      374
positive      9
negative     23
anger         0
disgust      22
fear          0
sadness       1
surprise      0
good          7
happy         2
dtype: int64
length      374
positive      9
negative     23
anger         0
disgust      22
fear          0
sadness       1
surprise      0
good          7
happy         2
dtype: int64
length      374
positive      9
negative     23
anger         0
disgust      22
fear          0
sadness       1
surprise      0
good          7
happy         2
dtype: int64
length      55
positive     2
negative     1
anger        0
disgust      0
fear         0
sadness      1
surprise     0
good         2
happy        0
dtype: int64
length      437
positive     10
negative      1
anger         0
disgust       1
fear          0
sadness       0
surprise      0
good          7
happy         3
dtype: int64
length      437
positive     10
negative      1
anger         0
disgust       1
fear          0
sadness       0
surprise      0
good          7
h

length      301
positive      9
negative     19
anger         0
disgust      12
fear          5
sadness       2
surprise      0
good          4
happy         5
dtype: int64
length      301
positive      9
negative     19
anger         0
disgust      12
fear          5
sadness       2
surprise      0
good          4
happy         5
dtype: int64
length      301
positive      9
negative     19
anger         0
disgust      12
fear          5
sadness       2
surprise      0
good          4
happy         5
dtype: int64
length      301
positive      9
negative     19
anger         0
disgust      12
fear          5
sadness       2
surprise      0
good          4
happy         5
dtype: int64
length      286
positive     11
negative     10
anger         1
disgust       9
fear          0
sadness       0
surprise      0
good          6
happy         5
dtype: int64
length      286
positive     11
negative     10
anger         1
disgust       9
fear          0
sadness       0
surprise      0
good   

length      420
positive      8
negative     18
anger         1
disgust      12
fear          1
sadness       4
surprise      0
good          8
happy         0
dtype: int64
length      130
positive      6
negative      6
anger         0
disgust       4
fear          1
sadness       1
surprise      0
good          6
happy         0
dtype: int64
length      362
positive     18
negative      1
anger         0
disgust       0
fear          0
sadness       1
surprise      0
good         17
happy         1
dtype: int64
length      267
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      267
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      267
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good   

length      233
positive      6
negative     18
anger         0
disgust      17
fear          1
sadness       0
surprise      1
good          4
happy         1
dtype: int64
length      233
positive      6
negative     18
anger         0
disgust      17
fear          1
sadness       0
surprise      1
good          4
happy         1
dtype: int64
length      305
positive     17
negative     20
anger         1
disgust      12
fear          1
sadness       6
surprise      0
good         13
happy         4
dtype: int64
length      333
positive     13
negative      7
anger         0
disgust       5
fear          1
sadness       1
surprise      0
good         11
happy         2
dtype: int64
length      333
positive     13
negative      7
anger         0
disgust       5
fear          1
sadness       1
surprise      0
good         11
happy         2
dtype: int64
length      132
positive      2
negative      3
anger         0
disgust       2
fear          1
sadness       0
surprise      0
good   

length      173
positive      6
negative     14
anger         0
disgust       5
fear          0
sadness       9
surprise      1
good          3
happy         2
dtype: int64
length      245
positive      2
negative     11
anger         0
disgust      10
fear          0
sadness       1
surprise      0
good          2
happy         0
dtype: int64
length      301
positive     11
negative     20
anger         0
disgust      12
fear          5
sadness       3
surprise      0
good          6
happy         5
dtype: int64
length      301
positive     11
negative     20
anger         0
disgust      12
fear          5
sadness       3
surprise      0
good          6
happy         5
dtype: int64
length      301
positive     11
negative     20
anger         0
disgust      12
fear          5
sadness       3
surprise      0
good          6
happy         5
dtype: int64
length      301
positive     11
negative     20
anger         0
disgust      12
fear          5
sadness       3
surprise      0
good   

...

length      183
positive      1
negative     10
anger         0
disgust       3
fear          0
sadness       7
surprise      0
good          1
happy         0
dtype: int64
length      346
positive      4
negative     12
anger         0
disgust      11
fear          1
sadness       0
surprise      0
good          3
happy         1
dtype: int64
length      282
positive      8
negative     14
anger         0
disgust      13
fear          1
sadness       0
surprise      1
good          4
happy         3
dtype: int64
length      654
positive     37
negative     39
anger         0
disgust      20
fear          3
sadness      16
surprise      0
good         35
happy         2
dtype: int64
length      142
positive     10
negative      9
anger         0
disgust       9
fear          0
sadness       0
surprise      0
good         10
happy         0
dtype: int64
length      288
positive      7
negative      6
anger         1
disgust       1
fear          1
sadness       3
surprise      0
good   

length      469
positive     31
negative     11
anger         0
disgust       6
fear          1
sadness       4
surprise      0
good         30
happy         1
dtype: int64
length      321
positive     17
negative      7
anger         0
disgust       3
fear          4
sadness       0
surprise      0
good         12
happy         5
dtype: int64
length      301
positive     10
negative     15
anger         1
disgust       7
fear          0
sadness       7
surprise      0
good          5
happy         5
dtype: int64
length      288
positive     23
negative     18
anger         0
disgust      15
fear          1
sadness       2
surprise      1
good         20
happy         2
dtype: int64
length      94
positive     2
negative     6
anger        0
disgust      6
fear         0
sadness      0
surprise     0
good         2
happy        0
dtype: int64
length      54
positive     3
negative     3
anger        0
disgust      1
fear         1
sadness      1
surprise     0
good         3
happy     

length      531
positive      2
negative      4
anger         0
disgust       4
fear          0
sadness       0
surprise      0
good          2
happy         0
dtype: int64
length      531
positive      2
negative      4
anger         0
disgust       4
fear          0
sadness       0
surprise      0
good          2
happy         0
dtype: int64
length      126
positive      0
negative      5
anger         2
disgust       2
fear          0
sadness       1
surprise      0
good          0
happy         0
dtype: int64
length      404
positive      9
negative     17
anger         0
disgust      13
fear          1
sadness       3
surprise      0
good          9
happy         0
dtype: int64
length      404
positive      9
negative     17
anger         0
disgust      13
fear          1
sadness       3
surprise      0
good          9
happy         0
dtype: int64
length      186
positive     11
negative      3
anger         0
disgust       2
fear          0
sadness       1
surprise      0
good   

length      254
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      254
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      254
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      254
positive      3
negative      0
anger         0
disgust       0
fear          0
sadness       0
surprise      0
good          3
happy         0
dtype: int64
length      374
positive     21
negative      4
anger         0
disgust       2
fear          1
sadness       1
surprise      0
good         18
happy         3
dtype: int64
length      239
positive      9
negative      4
anger         0
disgust       4
fear          0
sadness       0
surprise      0
good   

KeyboardInterrupt: 

In [ ]:
pip install tqdm

In [ ]:
from tqdm.notebook import tqdm
import time

# ---------------------------------------情感计算---------------------------------
start = time.time()   

# 创建 tqdm 对象
progress_bar = tqdm(total=len(weibo_df['新闻内容']))

# 模拟情感计算代码块
def apply_emotion_caculate(text):
    # 模拟情感计算的代码
    time.sleep(0.1)  # 这里用 sleep 模拟实际的计算时间
    progress_bar.update(1)  # 更新进度条
    return emotion_caculate(text)

# 使用 apply 方法并传递 tqdm 更新进度条
emotion_df = weibo_df['新闻内容'].apply(apply_emotion_caculate)

# 关闭 tqdm 进度条
progress_bar.close()

end = time.time()
print(end - start)
print(emotion_df.head())


# 输出结果，将原数据框与情感值数据框合并
output_df = pd.concat([weibo_df, emotion_df], axis=1)

# 保存结果到Excel文件
output_file_path = '全网新闻数据4.xlsx'
output_df.to_excel(output_file_path, index=False)
print(f"处理后的数据已保存到 {output_file_path}")



  0%|          | 0/86538 [00:00<?, ?it/s]

# 使用道德词典提取道德词

In [1]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import jieba     # 中文分词
import cmfd     # 中文道德词典

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.668 seconds.
Prefix dict has been built successfully.


In [ ]:
from moralstrength import lexicon_use
from moralstrength.moralstrength import estimate_morals

In [2]:
chn_moral = pd.read_csv(r'https://raw.githubusercontent.com/CivicTechLab/CMFD/main/cmfd_civictech.csv')

In [3]:
chn_moral

,chinese,foundation
0,同情,care
1,一臂之力,care
2,一见倾心,care
3,三个代表,care
4,上阵杀敌,care
...,...,...
6133,随和,general
6134,雅正,general
6135,雷打不动,general
6136,马马虎虎,general


In [4]:
moral_dict = chn_moral.groupby('foundation')['chinese'].apply(list).to_dict()

In [7]:
def moral_quantity(text):
    
    if isinstance(text, str):
        moral_word_total = 0
        moral_word = {}
        moral_num = {}

        for key in moral_dict.keys():
            moral_word[key] = []
        for word in jieba.cut(text):
            for key in moral_dict.keys():
                if word in moral_dict[key]:
                    moral_word[key].append(word)

        for key in moral_word.keys():
            moral_word_total += len(moral_word[key])
        if moral_word_total == 0:
            return None

        for key in moral_word.keys():
            moral_num[key] = len(moral_word[key]) / moral_word_total

    return moral_num

In [8]:
df_chn = pd.read_excel("全网新闻数据3.xlsx")

In [9]:
df_chn['chn_moral'] = df_chn['新闻内容'].apply(moral_quantity)
df_chn.head()

,date,新闻内容,城市,chn_moral
0,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,上海,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca..."
1,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,浙江,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca..."
2,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,河北,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca..."
3,2012 06 15,中国台湾 网 月日 消息 台湾 中国时报 报道 岛内 大学教授 假发票 真钱案 滚雪球 愈演...,台湾,"{'altr': 0.0, 'auth': 1.0, 'care': 0.0, 'dili'..."
4,2012 06 15,上海 试行 启运 港 退税 政策 通知 魇 灾吻 直辖市 计划单列市 财政厅 局 国家税务局...,上海,"{'altr': 0.0, 'auth': 0.8, 'care': 0.0, 'dili'..."


In [10]:
chn_moral_df = pd.DataFrame(columns=['altr', 'auth', 'care', 'dili', 'fair', 'general', 'libe', 'loya', 'mode', 'resi', 'sanc', 'wast'])

for dc in df_chn.index:
    if df_chn['chn_moral'][dc] == None:
        chn_moral_df.loc[len(chn_moral_df.index)] = [None] * 12
    else:
        chn_moral_df.loc[len(chn_moral_df.index)] = list(df_chn['chn_moral'][dc].values())
        
df_chn = pd.concat([df_chn, chn_moral_df], axis=1)
        
df_chn.head()

,date,新闻内容,城市,chn_moral,altr,auth,care,dili,fair,general,libe,loya,mode,resi,sanc,wast
0,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,上海,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca...",0.0,0.405405,0.054054,0.0,0.297297,0.0,0.0,0.216216,0.0,0.0,0.027027,0.0
1,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,浙江,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca...",0.0,0.405405,0.054054,0.0,0.297297,0.0,0.0,0.216216,0.0,0.0,0.027027,0.0
2,2012 06 12,中广网 唐山 月日 消息 汤 一亮 庄胜春据 中国 之声 新闻 晚 高峰 报道 日 上午 公...,河北,"{'altr': 0.0, 'auth': 0.40540540540540543, 'ca...",0.0,0.405405,0.054054,0.0,0.297297,0.0,0.0,0.216216,0.0,0.0,0.027027,0.0
3,2012 06 15,中国台湾 网 月日 消息 台湾 中国时报 报道 岛内 大学教授 假发票 真钱案 滚雪球 愈演...,台湾,"{'altr': 0.0, 'auth': 1.0, 'care': 0.0, 'dili'...",0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,2012 06 15,上海 试行 启运 港 退税 政策 通知 魇 灾吻 直辖市 计划单列市 财政厅 局 国家税务局...,上海,"{'altr': 0.0, 'auth': 0.8, 'care': 0.0, 'dili'...",0.0,0.800000,0.000000,0.0,0.150000,0.0,0.0,0.050000,0.0,0.0,0.000000,0.0


In [11]:
df_chn.to_excel("全网新闻数据5.xlsx", index=False)

<ipython-input-11-7281cdb308f5>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df_chn.to_excel("全网新闻数据5.xlsx", index=False)


# 提取叙事特征

In [ ]:
import re
import nltk
from gensim.models import Word2Vec

In [ ]:
import pandas as pd
import numpy as np
import glob,os
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle
import pickle
import h5py
import json
import matplotlib.pyplot as plt 

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback
from keras.models import model_from_json


# importing bokeh library for interactive dataviz
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [ ]:
conda install -c conda-forge scipy

In [ ]:
conda update -n base -c conda-forge conda

In [ ]:
# Initialize the Word2Vec model
model = gensim.models.Word2Vec(data2018_active_sep.tokens, vector_size=300, window=5, min_count=5, workers=4,sg=1,negative = 5)
# Train the Word2Vec model
model.train(tweets, total_examples=len(data2018_active_sep.Tweet_content), epochs=10)

def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size)) #这里实则是将300维数组降为1*300维了
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

## 提取每条文本的300维向量
wordvec_arrays = np.zeros((len(tweets), 300))
for i in range(len(tweets)):
    wordvec_arrays[i,:] = word_vector(tweets[i], 300)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

wordvec_arrays[0,:] = word_vector(tweets[0], 300)

wordvec_df.head()

In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm

# 读取数据
df = pd.read_excel('全网新闻数据5.xlsx')

# 分词处理
df['tokens'] = df['新闻内容']

# 初始化Word2Vec模型
model = Word2Vec(df['tokens'], vector_size=300, window=5, min_count=5, workers=4, sg=1, negative=5)

# 训练Word2Vec模型
model.train(df['tokens'], total_examples=len(df['新闻内容']), epochs=10)

# 定义提取单词向量的函数
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

# 提取每条文本的300维向量
wordvec_arrays = np.zeros((len(df['tokens']), 300))
for i, tokens in tqdm(enumerate(df['tokens'])):
    wordvec_arrays[i, :] = word_vector(tokens, 300)

# 创建包含Word2Vec向量的DataFrame
wordvec_df = pd.DataFrame(wordvec_arrays, columns=[f'word_vec_{i+1}' for i in range(300)])

# 将Word2Vec向量与原始数据框合并
df = pd.concat([df, wordvec_df], axis=1)

# 打印结果
print(df.head())


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm

# 读取数据
df = pd.read_excel('清洗后的新闻联播文本.xlsx')

# 分词处理
df['tokens'] = df['文本']

# 初始化Word2Vec模型
model = Word2Vec(df['tokens'], size=300, window=5, min_count=5, workers=4, sg=1, negative=5)

# 训练Word2Vec模型
model.train(df['tokens'], total_examples=len(df['文本']), epochs=10)

# 定义提取单词向量的函数
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

# 提取每条文本的300维向量
wordvec_arrays = np.zeros((len(df['tokens']), 300))
for i, tokens in tqdm(enumerate(df['tokens'])):
    wordvec_arrays[i, :] = word_vector(tokens, 300)

# 创建包含Word2Vec向量的DataFrame
wordvec_df = pd.DataFrame(wordvec_arrays, columns=[f'word_vec_{i+1}' for i in range(300)])

# 将Word2Vec向量与原始数据框合并
df = pd.concat([df, wordvec_df], axis=1)

# 打印结果
print(df.head())


In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm

# 读取数据
df = pd.read_excel('清洗后的新闻联播文本.xlsx')

# 分词处理
tokenizer = lambda x: x.split()  # 示例分词器，将每个文本以空格分割
df['tokens'] = df['文本'].apply(tokenizer)

# 初始化Word2Vec模型
model = Word2Vec(df['tokens'], size=300, window=5, min_count=5, workers=4, sg=1, negative=5)

# 训练Word2Vec模型
with tqdm(total=len(df['文本']), desc="Training Word2Vec") as pbar:
    model.train(df['tokens'], total_examples=len(df['文本']), epochs=10, callbacks=[lambda count, num_epochs: pbar.update(1)])

# 定义提取单词向量的函数
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

# 提取每条文本的300维向量
with tqdm(total=len(df['文本']), desc="Extracting Word Vectors") as pbar:
    wordvec_arrays = np.zeros((len(df['tokens']), 300))
    for i, tokens in enumerate(df['tokens']):
        wordvec_arrays[i, :] = word_vector(tokens, 300)
        pbar.update(1)

# 创建包含Word2Vec向量的DataFrame
wordvec_df = pd.DataFrame(wordvec_arrays, columns=[f'word_vec_{i+1}' for i in range(300)])

# 将Word2Vec向量与原始数据框合并
df = pd.concat([df, wordvec_df], axis=1)

# 打印结果
print(df.head())


# LDA

In [1]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis
from gensim import corpora, models
from gensim.models import CoherenceModel
from tqdm import tqdm

In [4]:
import pandas as pd
from gensim import corpora
import ast

# 读取数据
df = pd.read_excel('全网新闻数据3.xlsx')

# 确保新闻内容列是包含分词结果的列表
df['新闻内容'] = df['新闻内容'].apply(ast.literal_eval)

# 创建字典和语料库
dictionary = corpora.Dictionary(df['新闻内容'])  # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['新闻内容']]  # 根据分词结果创建语料库

# 打印字典和语料库以验证结果
print(dictionary)
print(corpus[:5])  # 仅打印前5个语料库内容


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
# 函数：计算一致性得分
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus=corpus,
                                id2word=dictionary,
                                num_topics=num_topics, 
                                passes=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
# 计算从2到40，每隔6个主题数量的一致性得分
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=df['分词'], start=2, limit=40, step=6)

# 绘制一致性得分
x = range(2, 40, 6)
plt.plot(x, coherence_values)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.legend(["coherence_values"], loc='best')
plt.show()


In [ ]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=13, id2word=dictionary, passes=15)


In [ ]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

In [ ]:
# 为每行新闻内容分配主题
 topics = lda_model.get_document_topics(bow)
    # 选择概率最高的主题
    return max(topics, key=lambda x: x[1])[0]

# 将主题分配到新闻内容
tqdm.pandas(desc="分配主题")
df['主题'] = df['分词'].progress_apply(get_topic)

# 保存结果到新的Excel文件
df.to_excel('新闻内容主题分配.xlsx', index=False)

print("处理完成，文件已保存为 '新闻内容主题分配.xlsx'")


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 合并数据

In [4]:
import pandas as pd

# 读取两个Excel文件
df1 = pd.read_excel('全网新闻数据4.xlsx')
df2 = pd.read_excel('全网新闻数据5.xlsx')

# 指定需要匹配的列
merge_columns = ['date', '新闻内容', '城市']

# 合并两个DataFrame
merged_df = pd.merge(df1, df2, on=merge_columns, how='outer')

# 保存合并后的结果到新的Excel文件
merged_df.to_excel('全网新闻数据7.xlsx', index=False)

print("合并完成，文件已保存为 '全网新闻数据7.xlsx'")


<ipython-input-4-a2c5b62a5270>:14: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  merged_df.to_excel('全网新闻数据7.xlsx', index=False)


合并完成，文件已保存为 '全网新闻数据7.xlsx'


In [5]:
#合并道德和情感进入面板数据
import pandas as pd

# 读取Excel文件
df = pd.read_excel('全网新闻数据7.xlsx')

# 指定需要合并的列
group_columns = ['date', '城市']

# 指定需要计算平均值的列
average_columns = [
    'length', 'positive', 'negative', 'anger', 'disgust', 'fear', 'sadness',
    'surprise', 'good', 'happy', 'altr', 'auth', 'care', 'dili', 'fair',
    'general', 'libe', 'loya', 'mode', 'resi', 'sanc', 'wast'
]

# 按指定列进行分组，并计算平均值
grouped_df = df.groupby(group_columns)[average_columns].mean().reset_index()

# 保存结果到新的Excel文件
grouped_df.to_excel('全网新闻数据8.xlsx', index=False)

print("处理完成，文件已保存为 '全网新闻数据8.xlsx'")


<ipython-input-5-a7609af68427>:20: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  grouped_df.to_excel('全网新闻数据8.xlsx', index=False)


处理完成，文件已保存为 '全网新闻数据8.xlsx'


In [11]:
import pandas as pd
import warnings

# 读取Excel文件
excel_path = "保证完成任务.xlsx"
df = pd.read_excel(excel_path)

# 定义一个函数来安全地解析日期，并跳过解析错误
def safe_parse_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='raise')
    except Exception as e:
        warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
        return pd.NaT

# 应用安全的日期解析函数
df['date'] = df['date'].apply(safe_parse_date)

# 去除解析失败的行
df = df.dropna(subset=['date'])

# 根据具体日期、城市和主题进行分组并统计数量
grouped_df = df.groupby([df['date'].dt.date, '城市', '主题']).size().reset_index(name='Count')

# 重新整理数据结构，将每个主题作为列
pivot_df = grouped_df.pivot_table(index=['date', '城市'], columns='主题', values='Count', fill_value=0).reset_index()

# 保存结果到新的Excel文件
output_excel_path = "城市-日期-主题.xlsx"
pivot_df.to_excel(output_excel_path, index=False)


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-11-2bb0bf1df761>:13: UserWarning: 无法解析日期: 0222 47 28，错误: month must be in 1..12: 0222 47 28, at position 0
  warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
<ipython-input-11-2bb0bf1df761>:13: UserWarning: 无法解析日期: 0322 47 28，错误: month must be in 1..12: 0322 47 28, at position 0
  warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
<ipython-input-11-2bb0bf1df761>:13: UserWarning: 无法解析日期: 0321 47 28，错误: month must be in 1..12: 0321 47 28, at position 0
  warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
<ipython-input-11-2bb0bf1df761>:30: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently ins

In [16]:
import pandas as pd
import warnings

# 读取Excel文件
excel_path = "全网新闻数据8.xlsx"
df = pd.read_excel(excel_path)

# 定义一个函数来安全地解析日期，并跳过解析错误
def safe_parse_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='raise').date()  # 仅提取年月日部分
    except Exception as e:
        warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
        return pd.NaT

# 应用安全的日期解析函数
df['date'] = df['date'].apply(safe_parse_date)

# 去除解析失败的行
df = df.dropna(subset=['date'])

# 保存转换后的结果到新的Excel文件
output_excel_path = "全网新闻数据9.xlsx"
df.to_excel(output_excel_path, index=False)

print(f"日期转换完成，结果保存到 {output_excel_path}")


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-16-26630e5f007d>:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_excel_path, index=False)


日期转换完成，结果保存到 全网新闻数据9.xlsx


In [20]:
#统计出现的报道的种类数量
import pandas as pd

# 读取Excel文件
excel_path = "全网新闻数据9.xlsx"
df = pd.read_excel(excel_path)

# 统计每行从第4列到第12列不为0的数字的数量
df['报道主题数量'] = df.iloc[:, 24:37].apply(lambda row: row[row != 0].count(), axis=1)

# 保存结果到新的Excel文件
output_excel_path = "全网新闻数据10.xlsx"
df.to_excel(output_excel_path, index=False)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-20-ef168729aabc>:13: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_excel_path, index=False)


In [21]:
#报道最多的主题
import pandas as pd

# 读取Excel文件
excel_path = "全网新闻数据10.xlsx"
df = pd.read_excel(excel_path)

# 获取第4列到第12列的最大值所在的列名
max_column_name = df.iloc[:, 24:37].idxmax(axis=1)

# 将最大值的列名写入第14列
df['报道最多的主题'] = max_column_name

# 保存结果到新的Excel文件
output_excel_path = "全网新闻数据11.xlsx"
df.to_excel(output_excel_path, index=False)


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-21-d57f9cf81315>:16: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_excel_path, index=False)


In [23]:
import pandas as pd
import warnings

# 读取Excel文件
excel_path = "命苦牛牛不惧困难所向披靡(1).xlsx"
df = pd.read_excel(excel_path)

# 定义一个函数来安全地解析日期，并跳过解析错误
def safe_parse_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='raise').date()  # 仅提取年月日部分
    except Exception as e:
        warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
        return pd.NaT

# 应用安全的日期解析函数
df['date'] = df['date'].apply(safe_parse_date)

# 去除解析失败的行
df = df.dropna(subset=['date'])

# 保存转换后的结果到新的Excel文件
output_excel_path = "犯罪率数据.xlsx"
df.to_excel(output_excel_path, index=False)

print(f"日期转换完成，结果保存到 {output_excel_path}")


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


日期转换完成，结果保存到 犯罪率数据.xlsx


<ipython-input-23-2678982a022e>:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_excel_path, index=False)


In [24]:
import pandas as pd

# 读取两个Excel文件
df1 = pd.read_excel('全网新闻数据11.xlsx')
df2 = pd.read_excel('犯罪率数据.xlsx')

# 指定需要匹配的列
merge_columns = ['date', '城市']

# 合并两个DataFrame
merged_df = pd.merge(df1, df2, on=merge_columns, how='outer')

# 保存合并后的结果到新的Excel文件
merged_df.to_excel('全网新闻数据12.xlsx', index=False)

print("合并完成，文件已保存为 '全网新闻数据12.xlsx'")


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-24-9965b97c034a>:14: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  merged_df.to_excel('全网新闻数据12.xlsx', index=False)


合并完成，文件已保存为 '全网新闻数据12.xlsx'


In [2]:
import pandas as pd

# 读取 Excel 文件
file_path = '全网新闻数据12.xlsx'
df = pd.read_excel(file_path)

# 确保列名正确
emotion_columns = ['anger', 'disgust', 'fear', 'sadness', 'surprise', 'good', 'happy']



# 计算每个城市的情绪列的平均值，并找到平均值最高的情绪
city_max_emotion = df.groupby('城市')[emotion_columns].mean().idxmax(axis=1)
city_max_emotion_mean = df.groupby('城市')[emotion_columns].mean().max(axis=1)

# 创建结果 DataFrame
result_df = pd.DataFrame({
    '城市': city_max_emotion.index,
    '最高平均值情绪': city_max_emotion.values,
    '最高平均值': city_max_emotion_mean.values
})

# 保存结果到新的 Excel 文件
result_df.to_excel('城市最高平均值情绪.xlsx', index=False)


<ipython-input-2-6b26c53c012a>:22: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('城市最高平均值情绪.xlsx', index=False)


In [3]:
import pandas as pd

# 读取 Excel 文件
file_path = '全网新闻数据12.xlsx'
df = pd.read_excel(file_path)

# 确保列名正确
emotion_columns = ['altr','auth','care','dili','fair','general','libe','loya','mode','resi','sanc','wast']



# 计算每个城市的情绪列的平均值，并找到平均值最高的情绪
city_max_emotion = df.groupby('城市')[emotion_columns].mean().idxmax(axis=1)
city_max_emotion_mean = df.groupby('城市')[emotion_columns].mean().max(axis=1)

# 创建结果 DataFrame
result_df = pd.DataFrame({
    '城市': city_max_emotion.index,
    '最高平均值道德': city_max_emotion.values,
    '最高平均值': city_max_emotion_mean.values
})

# 保存结果到新的 Excel 文件
result_df.to_excel('城市最高平均值道德.xlsx', index=False)


<ipython-input-3-8637f79ded48>:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('城市最高平均值道德.xlsx', index=False)


In [4]:
import pandas as pd

# 读取 Excel 文件
file_path = '全网新闻数据12.xlsx'
df = pd.read_excel(file_path)

# 确保列名正确
emotion_columns = ['中观经济', '交通事故', '亲属朋友', '体育竞技', '宏观经济', '微观经济', '政府倡议', '文化教育', '法院判决', '涉外报道', '社交网络', '自然灾害', '重大案件']



# 计算每个城市的情绪列的平均值，并找到平均值最高的情绪
city_max_emotion = df.groupby('城市')[emotion_columns].mean().idxmax(axis=1)
city_max_emotion_mean = df.groupby('城市')[emotion_columns].mean().max(axis=1)

# 创建结果 DataFrame
result_df = pd.DataFrame({
    '城市': city_max_emotion.index,
    '最高平均值主题': city_max_emotion.values,
    '最高平均值': city_max_emotion_mean.values
})

# 保存结果到新的 Excel 文件
result_df.to_excel('城市最高平均值主题.xlsx', index=False)


<ipython-input-4-a3e90057b397>:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('城市最高平均值主题.xlsx', index=False)


In [2]:
#合并道德和情感进入面板数据
import pandas as pd

# 读取Excel文件
df = pd.read_excel('搜狗新闻叙事结构测量.xlsx')

# 指定需要合并的列
group_columns = ['date', '城市']

# 指定需要计算平均值的列
average_columns = [
    '叙事容量', '叙事速度', '叙事迂回度'
]

# 按指定列进行分组，并计算平均值
grouped_df = df.groupby(group_columns)[average_columns].mean().reset_index()

# 保存结果到新的Excel文件
grouped_df.to_excel('全网新闻数据13.xlsx', index=False)

print("处理完成，文件已保存为 '全网新闻数据13.xlsx'")

<ipython-input-2-1fadbde76ca2>:19: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  grouped_df.to_excel('全网新闻数据13.xlsx', index=False)


处理完成，文件已保存为 '全网新闻数据13.xlsx'


In [4]:
import pandas as pd
import warnings

# 读取Excel文件
excel_path = "全网新闻数据13.xlsx"
df = pd.read_excel(excel_path)

# 定义一个函数来安全地解析日期，并跳过解析错误
def safe_parse_date(date_str):
    try:
        return pd.to_datetime(date_str, errors='raise').date()  # 仅提取年月日部分
    except Exception as e:
        warnings.warn(f"无法解析日期: {date_str}，错误: {e}")
        return pd.NaT

# 应用安全的日期解析函数
df['date'] = df['date'].apply(safe_parse_date)

# 去除解析失败的行
df = df.dropna(subset=['date'])

# 保存转换后的结果到新的Excel文件
output_excel_path = "全网新闻数据14.xlsx"
df.to_excel(output_excel_path, index=False)

print(f"日期转换完成，结果保存到 {output_excel_path}")


<ipython-input-4-94390f1a74b9>:24: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(output_excel_path, index=False)


日期转换完成，结果保存到 全网新闻数据14.xlsx


In [5]:
import pandas as pd

# 读取两个Excel文件
df1 = pd.read_excel('全网新闻数据12.xlsx')
df2 = pd.read_excel('全网新闻数据14.xlsx')

# 指定需要匹配的列
merge_columns = ['date', '城市']

# 合并两个DataFrame
merged_df = pd.merge(df1, df2, on=merge_columns, how='outer')

# 保存合并后的结果到新的Excel文件
merged_df.to_excel('全网新闻数据15.xlsx', index=False)

print("合并完成，文件已保存为 '全网新闻数据15.xlsx'")


<ipython-input-5-42e31fd7bb4c>:14: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  merged_df.to_excel('全网新闻数据15.xlsx', index=False)


合并完成，文件已保存为 '全网新闻数据15.xlsx'
